# Analyze Saved Model

9.27.21

#### Summary:
- Train (load) a model that was developed by using FedEM
- Check the nn class within the model
- Figure out how to work Dataloader
- Figure out differences between client, test, ensemble_learner -- hypothesis vs clients, linear weights


In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [3]:
# Arguments used for training
class args:
    
    def __init__(self):
        self.experiment = "cifar10"
        self.method = "FedEM"
        self.decentralized = False
        self.sampling_rate = 1.0
        self.input_dimension = None
        self.output_dimension = None
        self.n_learners= 3
        self.n_rounds = 10
        self.bz = 128
        self.local_steps = 1
        self.lr_lambda = 0
        self.lr =0.03
        self.lr_scheduler = 'multi_step'
        self.log_freq = 10
        self.device = 'cuda'
        self.optimizer = 'sgd'
        self.mu = 0
        self.communication_probability = 0.1
        self.q = 1
        self.locally_tune_clients = False
        self.seed = 1234
        self.verbose = 1
        self.save_path = 'weights/cifar/21_09_28_first_transfers/'
        self.validation = False
        
    
    def __iter__(self):
        for attr in dir(self):
            if not attr.startswith("__"):
                yield attr

args_ = args()

### Load the Saved Aggregator

In [7]:
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

print(weights)

[[0.00 0.35 0.65]
 [0.41 0.59 0.00]
 [0.42 0.09 0.49]
 [0.96 0.04 0.00]
 [0.62 0.00 0.38]
 [0.00 0.53 0.47]
 [0.04 0.44 0.52]
 [0.00 0.20 0.80]
 [0.00 0.73 0.27]
 [0.57 0.00 0.43]
 [0.15 0.00 0.85]
 [0.00 0.50 0.50]
 [0.64 0.11 0.25]
 [0.70 0.00 0.30]
 [0.31 0.00 0.69]
 [0.00 0.00 1.00]
 [0.01 0.89 0.10]
 [0.78 0.00 0.22]
 [1.00 0.00 0.00]
 [1.00 0.00 0.00]
 [0.71 0.00 0.29]
 [0.09 0.00 0.91]
 [0.00 0.15 0.85]
 [1.00 0.00 0.00]
 [0.71 0.00 0.29]
 [0.00 0.16 0.84]
 [0.00 0.00 1.00]
 [0.96 0.04 0.00]
 [0.77 0.00 0.23]
 [0.81 0.00 0.19]
 [0.00 0.62 0.38]
 [0.98 0.02 0.00]
 [0.68 0.00 0.32]
 [0.87 0.10 0.03]
 [0.00 0.36 0.64]
 [0.48 0.46 0.06]
 [1.00 0.00 0.00]
 [0.00 0.01 0.99]
 [0.00 1.00 0.00]
 [0.05 0.95 0.00]
 [0.68 0.32 0.00]
 [0.01 0.20 0.79]
 [0.86 0.14 0.00]
 [0.08 0.00 0.92]
 [0.00 1.00 0.00]
 [0.00 0.42 0.58]
 [0.03 0.00 0.97]
 [0.98 0.02 0.00]
 [0.01 0.00 0.99]
 [0.98 0.00 0.02]
 [0.00 0.00 1.00]
 [0.24 0.00 0.76]
 [0.00 0.00 1.00]
 [0.99 0.00 0.01]
 [0.80 0.00 0.20]
 [0.00 0.8

### Copy of Run Experiments

Try and set up an aggregator without going through the training process to observe the dimensionality of clients.

In [8]:
torch.manual_seed(args_.seed)

data_dir = get_data_dir(args_.experiment)

if "logs_root" in args_:
    logs_root = args_.logs_root
else:
    logs_root = os.path.join("logs", args_to_string(args_))

print("==> Clients initialization..")
clients = init_clients(
    args_,
    root_path=os.path.join(data_dir, "train"),
    logs_root=os.path.join(logs_root, "train")
)

print("==> Test Clients initialization..")
test_clients = init_clients(
    args_,
    root_path=os.path.join(data_dir, "test"),
    logs_root=os.path.join(logs_root, "test")
)

logs_path = os.path.join(logs_root, "train", "global")
os.makedirs(logs_path, exist_ok=True)
global_train_logger = SummaryWriter(logs_path)

logs_path = os.path.join(logs_root, "test", "global")
os.makedirs(logs_path, exist_ok=True)
global_test_logger = SummaryWriter(logs_path)

global_learners_ensemble = \
    get_learners_ensemble(
        n_learners=args_.n_learners,
        name=args_.experiment,
        device=args_.device,
        optimizer_name=args_.optimizer,
        scheduler_name=args_.lr_scheduler,
        initial_lr=args_.lr,
        input_dim=args_.input_dimension,
        output_dim=args_.output_dimension,
        n_rounds=args_.n_rounds,
        seed=args_.seed,
        mu=args_.mu
)


if args_.decentralized:
    aggregator_type = 'decentralized'
else:
    aggregator_type = AGGREGATOR_TYPE[args_.method]

aggregator =\
    get_aggregator(
        aggregator_type=aggregator_type,
        clients=clients,
        global_learners_ensemble=global_learners_ensemble,
        lr_lambda=args_.lr_lambda,
        lr=args_.lr,
        q=args_.q,
        mu=args_.mu,
        communication_probability=args_.communication_probability,
        sampling_rate=args_.sampling_rate,
        log_freq=args_.log_freq,
        global_train_logger=global_train_logger,
        global_test_logger=global_test_logger,
        test_clients=test_clients,
        verbose=args_.verbose,
        seed=args_.seed
    )


==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 161.78it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:34<00:00,  2.35it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


### Linear Combination of Weights

In [9]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

In [10]:
# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

In [11]:
# Determine here the different weights of models we desire to compare
model_weights = []

model_weights += [(1,0,0),(0,1,0),(0,0,1)] #single mixture
model_weights += [(0.5,0.5,0),(0.5,0,0.5),(0,0.5,0.5)] # half of 2 mixtures of 3
model_weights += [(1/3,1/3,1/3)] # Equally balanced from 3 mixtures

In [12]:
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

### Performance of Each of the Neural Networks
- Figure out how to bring the dataloader and load points
- Check accuracy for different combinations of hypotheses

In [13]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [14]:
torch.cuda.empty_cache()
# Pass subset of dataset through (10000 samples) and check accuracy on each model
bz = 2000 # < 47000
acc_dict = {}

# Select Random Subset of values
samples = random.sample(range(data_y.shape[0]),bz)

x_data = data_x[samples].to(device='cuda')
y_data = data_y[samples].to(device='cuda')

# Obtain Accuracy
for i in range(len(model_weights)):
    preds = models_test[i](x_data)
    preds_category = torch.argmax(preds,dim=1)
    acc_dict[model_weights[i]] = (preds_category == y_data).float().sum()/bz

acc_dict

{(1, 0, 0): tensor(0.8065, device='cuda:0'),
 (0, 1, 0): tensor(0.7225, device='cuda:0'),
 (0, 0, 1): tensor(0.8505, device='cuda:0'),
 (0.5, 0.5, 0): tensor(0.7970, device='cuda:0'),
 (0.5, 0, 0.5): tensor(0.8555, device='cuda:0'),
 (0, 0.5, 0.5): tensor(0.7910, device='cuda:0'),
 (0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333): tensor(0.8300, device='cuda:0')}

### Generate Evasion Attacks on Each of the Models
- Import victim NN and make modifications to work with the CIFAR model (any model)
- Make it work with any dataset that is given to it (tensor or dataloader)

In [15]:
# Write a Custom Class for Dataloader that has flexible batch size
class Custom_Dataloader:
    def __init__(self, x_data, y_data):
        self.x_data = x_data # Tensor + cuda
        self.y_data = y_data # Tensor + cuda
        
    def load_batch(self,batch_size,mode = 'test'):
        samples = random.sample(range(self.y_data.shape[0]),batch_size)
        out_x_data = self.x_data[samples].to(device='cuda')
        out_y_data = self.y_data[samples].to(device='cuda')
        
        return out_x_data, out_y_data


In [16]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

In [17]:
# Create adversary neural network from one of the test models
adv_nn = Adv_NN(trained_network=models_test[0], dataloader=dataloader)

In [18]:
# Generate attack parameters
atk_params = IFSGM_Params()
atk_params.target = 5
atk_params.x_val_min = torch.min(x_data).item()
atk_params.x_val_max = torch.max(x_data).item()
atk_params.batch_size = 10

In [19]:
# Perform attack on itself - IFGSM
adv_nn.i_fgsm(atk_params, print_info=True, mode = 'test')

---- FGSM Batch Size: 10 ----

Orig Target: [8, 2, 4, 6, 5, 3, 2, 9, 8, 2]
Orig Output: [8, 2, 4, 6, 3, 3, 2, 9, 8, 2]
ADV Output : [8, 6, 5, 5, 5, 5, 5, 5, 8, 5] 

Orig Loss  : -9.537945747375488
ADV Loss   : -4.195797443389893 

Orig Acc   : 0.9000000357627869
ADV Acc    : 0.30000001192092896


### Transfer Attack Single
- Utilize the transferer class across all of the different models and perform a sweep from different settings on how much attacks are transferable

In [20]:
# Make transferer and Assign model index
adv_idx = 0
victim_idxs = [0,1,2,3,4,5,6]

t1 = Transferer(models_list=models_test, dataloader=dataloader)
t1.generate_advNN(adv_idx)
t1.generate_victims(victim_idxs)

t1.generate_xadv(atk_type = "ifsgm")
t1.send_to_victims(victim_idxs)
t1.check_empirical_metrics(orig_flag = True)

In [21]:
t1.orig_acc_transfers

{0: tensor(0.8000, device='cuda:0'),
 1: tensor(0.7200, device='cuda:0'),
 2: tensor(0.8600, device='cuda:0'),
 3: tensor(0.8100, device='cuda:0'),
 4: tensor(0.8500, device='cuda:0'),
 5: tensor(0.7800, device='cuda:0'),
 6: tensor(0.8000, device='cuda:0')}

In [22]:
t1.orig_similarities

{0: tensor(1., device='cuda:0'),
 1: tensor(0.6300, device='cuda:0'),
 2: tensor(0.7100, device='cuda:0'),
 3: tensor(0.7500, device='cuda:0'),
 4: tensor(0.7800, device='cuda:0'),
 5: tensor(0.6500, device='cuda:0'),
 6: tensor(0.7100, device='cuda:0')}

In [23]:
t1.adv_acc_transfers

{0: tensor(0.2600, device='cuda:0'),
 1: tensor(0.6300, device='cuda:0'),
 2: tensor(0.6300, device='cuda:0'),
 3: tensor(0.4800, device='cuda:0'),
 4: tensor(0.4900, device='cuda:0'),
 5: tensor(0.5900, device='cuda:0'),
 6: tensor(0.5100, device='cuda:0')}

In [24]:
t1.adv_similarities

{0: tensor(1., device='cuda:0'),
 1: tensor(0.3100, device='cuda:0'),
 2: tensor(0.5600, device='cuda:0'),
 3: tensor(0.6400, device='cuda:0'),
 4: tensor(0.7200, device='cuda:0'),
 5: tensor(0.5000, device='cuda:0'),
 6: tensor(0.7000, device='cuda:0')}

In [25]:
t1.orig_target_hit

{0: tensor(0.0300, device='cuda:0'),
 1: tensor(0.0500, device='cuda:0'),
 2: tensor(0.1200, device='cuda:0'),
 3: tensor(0.0600, device='cuda:0'),
 4: tensor(0.0900, device='cuda:0'),
 5: tensor(0.1100, device='cuda:0'),
 6: tensor(0.1100, device='cuda:0')}

In [26]:
t1.adv_target_hit

{0: tensor(0.7300, device='cuda:0'),
 1: tensor(0.1800, device='cuda:0'),
 2: tensor(0.3900, device='cuda:0'),
 3: tensor(0.4600, device='cuda:0'),
 4: tensor(0.5300, device='cuda:0'),
 5: tensor(0.3600, device='cuda:0'),
 6: tensor(0.4900, device='cuda:0')}

### Check Empirical Metrics
- Size of input gradient - across data distribution across all victim NN
- Gradient Alignment - Between the surrogate and each of the victim NN
- Variance of loss - Just for the surrogate

In [27]:
t1.check_empirical_metrics()

In [28]:
t1.metric_variance

tensor(11.5668, device='cuda:0', grad_fn=<SubBackward0>)

In [29]:
t1.metric_alignment

{0: tensor(0.),
 1: tensor(1.4098),
 2: tensor(1.3749),
 3: tensor(1.1213),
 4: tensor(1.1321),
 5: tensor(1.3215),
 6: tensor(1.1811)}

In [30]:
t1.metric_ingrad

{0: tensor(0.0710),
 1: tensor(0.2525),
 2: tensor(0.1249),
 3: tensor(0.0486),
 4: tensor(0.0417),
 5: tensor(0.0477),
 6: tensor(0.0296)}